<a href="https://colab.research.google.com/github/Ahmed11Raza/Assignments-PIAIC-/blob/main/Hello_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
%pip install -qU langchain-pinecone langchain-google-genai

In [53]:

from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [55]:
import time

index_name = "online-rag-project"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [72]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [73]:
vector= embeddings.embed_query("hello!world")


In [28]:
vector

[0.058485664427280426,
 -0.03927433490753174,
 -0.05723588541150093,
 -0.037421129643917084,
 0.019745292142033577,
 -0.022222531959414482,
 0.020639382302761078,
 -0.020429447293281555,
 0.010109051130712032,
 -0.003625837853178382,
 0.04512795805931091,
 0.036301854997873306,
 -0.019109489396214485,
 -0.053597044199705124,
 0.009051880799233913,
 -0.032336026430130005,
 0.00493299774825573,
 -0.005479874089360237,
 -0.007608703337609768,
 -0.009110097773373127,
 -0.018590562045574188,
 0.004686904139816761,
 -0.03870522230863571,
 -0.013001742772758007,
 0.025494320318102837,
 0.014942537993192673,
 -0.013410160318017006,
 -0.08515772223472595,
 0.026391159743070602,
 0.05883677676320076,
 -0.014418897219002247,
 0.021780921146273613,
 -0.033650271594524384,
 -0.00044796615839004517,
 0.02060868591070175,
 -0.03303968533873558,
 0.026654869318008423,
 0.019912496209144592,
 -0.015876516699790955,
 -0.01459532417356968,
 0.023259397596120834,
 -0.11662805080413818,
 -0.006988630164414

In [74]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [75]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [76]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [77]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['b1f64f9b-cbf4-4ba0-9193-be37593909cb',
 '15cbd13d-188b-443c-8cc3-39b9481e2aaa',
 '2d793b80-763e-4515-a220-6f7a61daf345',
 'd0428432-84d9-4462-81c0-e5090076faac',
 '5c871101-5a6c-4262-8e2b-958bffab0c04',
 'f978f229-f235-45db-973d-0f64117e86f8',
 '5f2f6c18-e692-4fce-b0fd-c46f5e678c1f',
 '5db3f454-4166-4142-b05c-b9d3e989e253',
 '52227cdf-eb3b-4d69-a551-6ff771dba522',
 'cf44bcc1-063a-457e-a061-444841fb86c3']

In [78]:
len(documents)

10

In [79]:
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)


['e20bc077-cc4d-4e4c-b3f3-52eee8eedd72',
 'ef814664-e89c-4d02-a785-1a084a2a7895',
 '495ff859-c157-4ce0-afc9-369241d5ff72',
 'e42c68b1-fc7f-4939-ad40-1e27c81f0c92',
 'dd78fd3b-5dcb-4a31-ac3b-307609c06e11',
 '0d322946-962c-4d62-98e0-00b183818d0f',
 '57fe4526-d49c-4ff7-b558-c1be0d326c45',
 '40b3e076-ec0a-425f-b197-7fe871bf9f79',
 '29a9cb9f-82a7-4893-8e04-3900e7697e56',
 'b922ec96-ec9e-473d-8985-de25b5469a9f']

In [80]:
#Data Reterive

results = vector_store.similarity_search(
    "Hello! World",
    k=2, #top 2 properties
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

#Data Reterive


* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]


In [81]:
results = vector_store.similarity_search_with_score(
    "Hello!World"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.577636] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* [SIM=0.577636] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* [SIM=0.577081] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* [SIM=0.577081] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [82]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [86]:
def answer_to_user(query:str):
  # Vector Search
 vector_results = vector_store.similarity_search(query,k=2)
 print(len(vector_results))

 #Pass to Model Vector Result + user query
 final_answer = llm.invoke(f"Answer this query:{query},Here are some refrence to answer{vector_results}")

 return final_answer


In [91]:
answer = answer_to_user("Hello!World")

2


In [92]:
answer.content


'The provided documents are identical tweets announcing the creation of an exciting new project using LangChain.  The tweets encourage viewers to check it out.  There is no further information available from these documents.\n'